### python과 mysql을 연동 
1. 외부의 라이브러리 설치 (pymysql)
2. 라이브러리 로드 
3. sql 서버의 정보를 등록하여 접속 
4. sql 쿼리문을 작성하여 sql 서버에 쿼리문을 요청 
5. 쿼리문에 따라서 응답 메시지 출력

In [ ]:
# 라이브러리 설치 
!pip install pymysql

In [2]:
# 라이브러리 로드 
import pymysql

In [3]:
# sql 서버의 정보를 입력 -> 접속
# 내 컴퓨터에 연결 
_db = pymysql.connect(
    host = '127.0.0.1', 
    port = 3306, 
    user = 'root', 
    password = '1234', 
    db = 'ezen'
)

In [4]:
## Cursor 생성 
# DictCursor로 cursor 생성 
cursor = _db.cursor(pymysql.cursors.DictCursor)
# 기본 값 사용
cursor2 = _db.cursor()